In [ ]:
import pandas as pd

# 데이터 로드 (예시)
df = pd.read_csv('factory_data.csv')

# 1. 서울에서 공장이 가장 많은 구는 어디야?
구별_공장수 = df.groupby('구')['공장관리번호'].count().sort_values(ascending=False)
print(f"공장이 가장 많은 구: {구별_공장수.index[0]} ({구별_공장수.iloc[0]}개)")

# 2. 강동구에 있는 공장 중 '임대' 공장은 얼마나 돼?
강동구_임대 = df[(df['구'] == '강동구') & (df['소유구분'] == '임대')]
print(f"강동구 임대 공장 수: {len(강동구_임대)}개")

# 3. 최근 5년간 서울 전체 공장 등록 건수 추이를 보여줘
df['등록년도'] = pd.to_datetime(df['등록일자']).dt.year
최근5년 = df['등록년도'].max() - 4
연도별_등록수 = df[df['등록년도'] >= 최근5년].groupby('등록년도')['공장관리번호'].count()
print("최근 5년 등록 건수 추이:")
print(연도별_등록수)

# 4. 지난 20년간 서울에서 공장 수가 가장 많이 증가한 5개 구를 알려줘
최근20년 = df['등록년도'].max() - 19
df_20년전 = df[df['등록년도'] < 최근20년].groupby('구')['공장관리번호'].count()
df_최근 = df[df['등록년도'] >= 최근20년].groupby('구')['공장관리번호'].count()
증가량 = (df_최근 - df_20년전.reindex(df_최근.index, fill_value=0)).sort_values(ascending=False).head(5)
print("공장 수 증가 상위 5개 구:")
print(증가량)

# 5. 새로 생긴 공장들이 요즘 어느 지역에 많이 몰려 있어?
최근3년 = df['등록년도'].max() - 2
최근_공장 = df[df['등록년도'] >= 최근3년].groupby('구')['공장관리번호'].count().sort_values(ascending=False).head(5)
print("최근 3년 신규 공장이 많은 지역:")
print(최근_공장)

# 6. 어느 구가 제조업 중심지처럼 보이는지 알 수 있을까?
구별_분석 = df.groupby('구').agg({
    '공장관리번호': 'count',
    '직원수': 'sum',
    '공장면적': 'sum'
}).rename(columns={'공장관리번호': '공장수'})
구별_분석['종합점수'] = (구별_분석['공장수'] * 0.4 + 
                        구별_분석['직원수'] * 0.3 + 
                        구별_분석['공장면적'] * 0.3)
제조업_중심지 = 구별_분석.sort_values('종합점수', ascending=False).head(5)
print("제조업 중심지로 보이는 구:")
print(제조업_중심지)

# 7. 공장이지만 주거지와 가까워서 문제가 될 만한 곳이 있는지 찾을 수 있어?
# (주거지역 정보가 있다고 가정)
주거지근접_공장 = df[(df['주거지역거리'] < 500) & (df['업종'].isin(['화학', '금속', '섬유']))]
print(f"주거지 근접 우려 공장: {len(주거지근접_공장)}개")
print(주거지근접_공장.groupby('구')['공장관리번호'].count())

# 8. 여성 직원 비율이 높은 공장은 어떤 업종이 많아?
df['여성비율'] = (df['여성직원수'] / df['직원수']) * 100
여성비율_높은_공장 = df[df['여성비율'] >= 50]
업종별_여성비율 = 여성비율_높은_공장.groupby('업종')['공장관리번호'].count().sort_values(ascending=False)
print("여성 직원 비율이 높은 업종:")
print(업종별_여성비율.head(10))

# 9. 공장 면적 대비 직원 수가 많은 공장과 적은 공장 상위 5개씩 알려줘
df['면적당직원수'] = df['직원수'] / df['공장면적']
많은_공장 = df.nlargest(5, '면적당직원수')[['공장명', '구', '면적당직원수', '직원수', '공장면적']]
적은_공장 = df.nsmallest(5, '면적당직원수')[['공장명', '구', '면적당직원수', '직원수', '공장면적']]
print("면적 대비 직원 수가 많은 공장 TOP 5:")
print(많은_공장)
print("\n면적 대비 직원 수가 적은 공장 TOP 5:")
print(적은_공장)

# 10. 생산품 기준으로 규모가 큰 공장들이 어떤 제품을 만드는지 궁금해
생산품별_규모 = df.groupby('생산품').agg({
    '공장관리번호': 'count',
    '공장면적': 'sum',
    '직원수': 'sum'
}).rename(columns={'공장관리번호': '공장수'})
생산품별_규모 = 생산품별_규모.sort_values('공장면적', ascending=False).head(10)
print("규모가 큰 생산품 종류:")
print(생산품별_규모)

# 11. 외국인 근로자가 많은 공장은 어디야? 업종 특징도 알려줘
df['외국인비율'] = (df['외국인근로자수'] / df['직원수']) * 100
외국인_많은_공장 = df.nlargest(20, '외국인근로자수')
print("외국인 근로자가 많은 공장:")
print(외국인_많은_공장[['공장명', '구', '업종', '외국인근로자수', '외국인비율']])
print("\n외국인 근로자가 많은 업종:")
print(df.groupby('업종')['외국인근로자수'].sum().sort_values(ascending=False).head(10))